<a href="https://colab.research.google.com/github/audiracmichelle/huggingface_workshop/blob/main/huggingface_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Face Workshop

## Part 1 — What is Hugging Face?
(Explain what the Hub, models, datasets, and libraries are.)

In [1]:
from datasets import load_dataset



## Part 2 — Downloading Datasets
(Code cells using `datasets` library.)



## Part 3 — Downloading Models with Transformers
(Code cells using `transformers` library.)



## Part 4 — Running LLMs Locally (Ollama & vLLM)
(Notes and optional code snippets showing usage.)